# Урок 3. Логистическая регрессия. Log Loss

In [1]:
# подключим необходимые библиотеки

import numpy as np
import matplotlib.pyplot as plt
import copy

from sklearn import datasets

%matplotlib inline

### 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Можно сделать переобозначение меток класса на -1 и +1. И в этом случае можно использовать функцию ошибок:
$$-\sum^{l}_{i=1} \text{ln}(1 + exp(\left \langle w,x_{i} \right \rangle))$$

либо для меток класса 0 и 1 использовать:
$$ -\sum^{l}_{i=1} (y_{i} \text{ln}\frac{1}{1 + exp(-\left \langle w,x_{i} \right \rangle)} + (1 - y_{i})\text{ln} \frac{exp(-\left \langle w,x_{i} \right \rangle)}{1 + exp(-\left \langle w,x_{i} \right \rangle)}).$$

В эти двух случаях мы имеем экспоненциальные функции, значения которых больше 0. А сложение с 1 под логарифмом или в знаменателе не даст 0.

In [2]:
# функцию в этом случае можно изменить на более простую
# loss = -1.0 / m * np.log(1 + np.exp(np.dot(w.T, X)))

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [3]:
"""
 Вспомогательные функции

"""

# Сигмоида
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


# функция потерь
def log_loss(w, X, y):
    m = X.shape[1]

    # используем функцию сигмоиды, написанную ранее
    A = sigmoid(np.dot(w.T, X)) 
    
    # вероятность отнесения объекта к классу "+1"
    loss = -1.0 / m * np.sum(y * np.log(A) + (1 - y) * np.log(1 - A))
    grad = 1.0 / m * np.dot(X, (A - y).T)
    
    return loss, grad


# Градиентный спуск
def optimize(w, X, y, n_iterations, alpha):
    # потери будем записывать в список для отображения в виде графика
    losses = []
    
    for i in range(n_iterations):        
        loss, grad = log_loss(w, X, y)
        w -= alpha * grad

        losses.append(loss)
        
    return w, losses

In [4]:
 # Сгенерируем датасет
classes = datasets.make_classification(n_samples=1000,
                                       n_features=50,
                                       n_informative=20,
                                       n_redundant=0,
                                       n_classes=2,
                                       random_state=1)
 
 
# перемешаем датасет
np.random.seed(12)
shuffle_index = np.random.permutation(classes[0].shape[0])
X, y = classes[0][shuffle_index], classes[1][shuffle_index]

# Транспонируем матрицы
X_tr = X.T
y_tr = y.reshape(1, y.shape[0])

y_tr

array([[1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 

In [5]:
# иницилизируем начальный вектор весов
w0 = np.zeros((X_tr.shape[0], 1))

n_iterations = 10000
alpha = 0.005

w, losses = optimize(w0, X_tr, y_tr, n_iterations, alpha)



print(f"Итоговый вектор весов w: {w}")

Итоговый вектор весов w: [[ 0.0826704 ]
 [-0.14852567]
 [ 0.11721495]
 [-0.46688234]
 [ 0.20198077]
 [ 0.01000646]
 [ 0.08826596]
 [ 0.06418002]
 [-0.09234752]
 [-0.22713773]
 [ 0.00658675]
 [ 0.13203703]
 [ 0.06560031]
 [-0.04347638]
 [-0.10568227]
 [-0.06823765]
 [-0.24657751]
 [ 0.09280656]
 [-0.05160206]
 [ 0.05680034]
 [-0.28848955]
 [-0.07884919]
 [-0.29207902]
 [ 0.06885236]
 [ 0.09987036]
 [ 0.10915652]
 [-0.03110719]
 [-0.03303839]
 [-0.10287925]
 [ 0.45402388]
 [-0.44826863]
 [-0.0073087 ]
 [-0.01976011]
 [ 0.37259946]
 [ 0.00847365]
 [-0.03747964]
 [ 0.12162821]
 [ 0.03771362]
 [-0.03894745]
 [-0.04657441]
 [ 0.13890005]
 [ 0.00673495]
 [-0.3913916 ]
 [-0.06742779]
 [-0.11021755]
 [ 0.08828855]
 [ 0.14209093]
 [-0.0407526 ]
 [ 0.13632125]
 [-0.27836117]]


### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [6]:
def calc_pred_proba(w, X):
    A = sigmoid(np.dot(w.T, X))
    return A

print(calc_pred_proba(w, X_tr))


[[9.40127463e-01 8.45359668e-03 2.30236065e-02 2.77590704e-01
  8.64877046e-01 5.30390549e-01 9.14795685e-01 9.72643767e-01
  5.82102674e-02 6.34329881e-01 2.23560992e-02 9.76713858e-01
  3.43760883e-02 1.07723069e-02 1.85948545e-03 2.50695673e-02
  9.88757756e-01 9.44247610e-01 5.68119826e-01 1.15242532e-01
  2.51426111e-01 2.32142600e-01 9.92164896e-01 2.50717288e-01
  9.24932403e-01 4.97465301e-02 7.34682054e-01 9.97696838e-01
  8.59871239e-01 2.14806550e-01 1.46746736e-04 9.62361364e-01
  3.06872108e-01 2.07973935e-02 1.75692613e-01 9.98723804e-01
  6.15755197e-01 8.09481242e-02 9.93949926e-01 5.24323712e-01
  4.96708973e-02 6.84695988e-01 3.06196981e-02 2.66069046e-02
  4.36996554e-02 9.71891492e-01 2.85223382e-01 9.05087022e-01
  1.40218460e-02 2.46054349e-02 9.86992384e-01 1.47325814e-02
  9.95425545e-01 9.86335273e-01 2.83917595e-01 7.29667839e-01
  9.97013794e-01 9.90189143e-01 1.43376190e-01 9.84275181e-01
  9.49051594e-01 3.78637452e-02 3.41606853e-01 9.65873368e-01
  1.8112

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [7]:
# Функция для выполения предсказаний
def predict(w, X):
    
    m = X.shape[1]
    
    y_predicted = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X))
    
    # За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(A.shape[1]):
        if (A[:,i] > 0.5): 
            y_predicted[:, i] = 1
        elif (A[:,i] <= 0.5):
            y_predicted[:, i] = 0
    
    return y_predicted

In [8]:
y_pred = predict(w, X_tr)
y_pred

array([[1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
        0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
        0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1.,
        0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0.,
        0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
        0., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 

### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [9]:
# В качестве меры точности возьмем долю правильных ответов
accuracy = 100.0 - np.mean(np.abs(y_pred - y_tr)*100.0)
print(f"Точность: {accuracy:.3f}")

Точность: 86.800


In [10]:
import pandas as pd


# Функция расчета метрик
def confusion_matrix_1(y, y_pred):
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(y)):
        tp += 1 if y[i] == y_pred[i] and y[i] == 1 else 0
        tn += 1 if y[i] == y_pred[i] and y[i] == 0 else 0
        fp += 1 if y[i] != y_pred[i] and y[i] == 0 else 0
        fn += 1 if y[i] != y_pred[i] and y[i] == 1 else 0
   
    matrix = pd.DataFrame({'y=1': [tp, fp],
                          'y=0': [fn, tn]}, 
                          index=['y_pred=1', 'y_pred=0'])
    
    return matrix, tp, tn, fp, fn

In [11]:
# Для тестирования посчитаем метрики при помощи библиотек
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print(f"f1 = {f1_score(y_tr[0], y_pred[0])}, prec = {precision_score(y_tr[0], y_pred[0])}, recall = {recall_score(y_tr[0], y_pred[0])}")
confusion_matrix(y_tr[0], y_pred[0])

f1 = 0.8639175257731959, prec = 0.8933901918976546, recall = 0.8363273453093812


array([[449,  50],
       [ 82, 419]])

In [12]:
matrix, tp, tn, fp, fn = confusion_matrix_1(y_tr[0], y_pred[0])

prec = tp/(tp+fp)
recall = tp/(tp+fn)
f1= 2 * prec * recall / (prec + recall)

print(f"f1 = {f1}, prec = {prec}, recall = {recall}")
matrix

f1 = 0.8639175257731959, prec = 0.8933901918976546, recall = 0.8363273453093812


,y=1,y=0
y_pred=1,419,82
y_pred=0,50,449


### 6. Могла ли модель переобучиться? Почему?

Модель могла переобучиться в силу специфики логарифмической функции ошибок, которая упирается в бескоечности не имеет минимумов. Модель бесконечно будет корректировать веса уменьшая ошибку и все больше подгонять ответ под данные с увеличением количесвта итераций или уменьшения скорости обучения.